# Neural Network Weight Optimisation Using Cultural Algorithms

## Tensorflow and PyGAD

### Importing Libararies

In [124]:
import sklearn
import pandas as pd
import pygad
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from pygad.kerasga import KerasGA
from sklearn.metrics.pairwise import cosine_similarity

### Configuring GPU

In [125]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [126]:
df = pd.read_csv('processed_data.csv')

In [127]:
df = df.drop('Unnamed: 0',axis = 1)

In [128]:
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,target
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0


In [129]:
df = pd.get_dummies(df,columns = ['Education'],drop_first = True)

### Loading and Preprocessing the Data

In [130]:
X = df.drop('target',axis = 1)
y = df['target']

In [131]:
len(X.columns)

12

### Splitting it into train and test and converting target into categorical data

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    test_size=0.33, 
                                                    random_state=42,
                                                    shuffle = True)

In [133]:
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)

### Defining a basic tensorflow ANN 

In [134]:
model = Sequential([tf.keras.layers.Dense(12,input_shape = (12,),activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(16, activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(2,activation = 'softmax')
                   ])

In [135]:
model.compile(optimizer = 'adam', metrics = ['accuracy'])

### Fitness function for the genetic algorithm

In [136]:
def fitness_func(solution, sol_idx):
    global X_train, y_train, keras_ga, model
    
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = solution)
    
    model.set_weights(weights = model_weights_matrix)
    
    predictions = model.predict(X_train)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    solution_fitness = 1.0/(loss(y_train,predictions).numpy() + 0.0000000001)
    
    return solution_fitness

### Acceptance Function for Cultural Algorithm

In [138]:
def acceptance_function(population, fitness):
    belief_space = population[fitness.index(max(fitness))]
    return belief_space

### Influence Function for Cultural Algorithm

In [139]:
def influence_function(belief_space,population):
    sim = cosine_similarity(belief_space.reshape(1,-1),population)
    return sim

### Select the solutions which have highest similarity to the belief space

In [140]:
def parent_selection(sim,population):
    # Get the indices of the two highest values using argsort
    highest_indices = np.argsort(sim[0])[-2:]

    # Get the values at the indices using fancy indexing
    highest_values = sim[0][highest_indices]
    parent_1 = population[highest_indices[0]]
    parent_2 = population[highest_indices[1]]
    return parent_1,parent_2

### Apply Genetic Operators 

In [141]:
def crossover(parent1, parent2):
    """
    Performs a crossover operation between two parents, producing two new children.

    Args:
        parent1 (numpy.ndarray): The first parent.
        parent2 (numpy.ndarray): The second parent.

    Returns:
        A tuple of two children produced by crossover.
    """
    # Determine the length of the parents and the crossover point.
    length = parent1.shape[0]
    crossover_point = np.random.randint(1, length - 1)

    # Create the first child by combining the parents.
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

    # Create the second child by combining the parents in reverse order.
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    return child1, child2

In [142]:
def mutation(array):
    """
    Adds a randomly generated value to 15% of the elements in a numpy array.

    Args:
        array (numpy.ndarray): The input array.

    Returns:
        A numpy array with mutations applied.
    """
    # Determine the number of elements to mutate.
    num_mutations = int(0.15 * array.size)

    # Generate random indices for the elements to mutate.
    mutation_indices = np.random.choice(array.size, size=num_mutations, replace=False)

    # Add a random value to the selected elements.
    array[mutation_indices] += np.random.randn(num_mutations)

    return array

### Keep the best members in the population

In [143]:
def prune_population(population,child1, child2):
    population = np.vstack((population, child1))
    population = np.vstack((population,child2))
# Calculate the fitness of each solution in the population.
    fitness_values = np.zeros((population.shape[0],))
    for i in range(population.shape[0]):
        fitness_values[i] = fitness_func(population[i], i)

    # Sort the population by fitness.
    sorted_indices = np.argsort(fitness_values)
    sorted_population = population[sorted_indices]

    # Return the top 10 elements.
    return sorted_population[-10:]

In [144]:
def cultural_algorithm(population):
    
    fitness = []
    for idx, solution in enumerate(population):
        fitness.append(fitness_func(solution,idx))
    
    belief_space = acceptance_function(population,fitness)
    similarity_score = influence_function(belief_space,population)
    parent1,parent2 = parent_selection(sim,population)
    child1,child2 = crossover(parent1, parent2)
    child1 = mutation(child1)
    child2 = mutation(child2)
    population = prune_population(population,child1, child2)
    return population

In [165]:
def set_final_weights(population):
    fitness = []
    for idx, solution in enumerate(population):
        fitness.append(fitness_func(solution,idx))
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = population[fitness.index(max(fitness))])
    
    model.set_weights(weights = model_weights_matrix)

In [145]:
# Get the shapes of the model's weights
weights_shapes = [w.shape for w in model.get_weights()]

# Compute the total number of weights
num_weights = np.sum([np.prod(s) for s in weights_shapes])

In [158]:
population = np.random.uniform(size = (10,num_weights))

In [159]:
for i in range(3):
    population = cultural_algorithm(population)

92/92 [==============================] - 0s 369us/step


In [164]:
model.evaluate(X_test,y_test)

45/45 [==============================] - 0s 447us/step - loss: 0.0000e+00 - accuracy: 0.9437


[0.0, 0.9437109231948853]